# Treinamento de uma rede neural Feed Foward para previsão de consumo de cimento 

## Arquitetura 

Consideramos cada estado, em cada mês de 1991 até 2019, como uma entrada separada do modelo

### Carregando os dados

Utilizamos o módulo `load_data` com os métodos `get_df` e `get_consumo`

In [6]:
def init ():

  global df, anomes

  # variaveis auxiliares
  anos = [str(i) for i in range(1991, 2020)]
  meses = ["{:02d}".format(i) for i in range(1,13)] # colocamos os 0 a esquerda se precisar
  anomes = []
  n_estados = 27

  for ano in anos:
    anomes += [str(ano) + i for i in meses]

  df = [[i] for i in range(1,28)] * len(anomes)

# funcao que pega um pandas dataframe e coloca os valor no array que sera usado como entrada para o  modelo
def df_to_list(indicador, array):

  global anomes
  
  index = 0 

  for coluna in anomes:

    for linha in indicador[coluna]:

      array[index] = array[index] + [linha]
      index += 1
  
  return array

# Carrega os dados de consumo de cimento 
# do PATH considerando a 
def get_consumo (path : str, var : float) -> list :

  import pandas as pd

  init()

  global anomes

  consumo = pd.read_csv(path + "consumo.csv")

  anomes = ["199012"] + anomes

  tendencia = []

  # anomes de 19901 ate 201912
  for index  in range(1,len(anomes)):

    # estados...
    for estado in consumo.index:

      tend = 1 - (consumo[anomes[index]][estado]/consumo[anomes[index - 1]][estado])

      if tend < - var:
        tendencia += [[1., 0., 0.]]

      elif tend <= var:
        tendencia += [[0., 1., 0.]]

      else:
        tendencia += [[0., 0., 1.]]

  anomes = anomes[1:]

  return tendencia

def get_df (path : str):

  import pandas as pd

  init()

  global df

  # PIB a precos constantes
  pib_cte = pd.read_csv(path + "pib_precos_constantes.csv")
  df = df_to_list(pib_cte, df)

  # PIB a precos de mercado corrente 
  pib_pm = pd.read_csv(path + "pib_precos_mercado_corrente.csv")
  df = df_to_list(pib_pm, df)

  # PIB a precos de mercado corrente 
  pib_pc = pd.read_csv(path + "pib_per_capita.csv")
  df = df_to_list(pib_pc, df)

  # PIB a precos de mercado corrente 
  pib_cc = pd.read_csv(path + "pib_construcao.csv")
  df = df_to_list(pib_cc, df)

  pop = pd.read_csv(path + "populacao.csv")
  df = df_to_list(pop, df)

  ipca = pd.read_csv(path + "IPCA.csv")
  df = df_to_list(ipca, df)

  incc = pd.read_csv(path + "INCC.csv")
  df = df_to_list(incc, df)

  selic = pd.read_csv(path + "SELIC.csv")
  df = df_to_list(selic, df)

  # IDH Renda
  idh_renda = pd.read_csv(path + "idh_renda.csv")
  df = df_to_list(idh_renda, df)

  # IDH Saúde
  idh_saude = pd.read_csv(path + "idh_saude.csv")
  df = df_to_list(idh_saude, df)

  # IDH Educação
  idh_educacao = pd.read_csv(path + "idh_educacao.csv")
  df = df_to_list(idh_educacao, df)

  return df

In [7]:
# Parâmetros

PATH = "../dados/anualizados/csv/"
VAR = 0.01

In [8]:
# Dados do consumo 
# from load_data import get_consumo 
tendencia = get_consumo(PATH, VAR)

In [9]:
# Dados de entrada 
# from load_data import get_df 
df = get_df(PATH)

## Treinamento do modelo

Vamos treinar uma rede neural utilizando a biblioteca TensorFlow

Dividimos os dados em:
- 70%: treino
- 30%: teste

In [10]:
# Importando as libs

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [11]:
# separando conjuntos de treino e teste (70% e 30%)
X_train, X_test, y_train, y_test = train_test_split(df, tendencia,test_size=0.2)

#### Definindo o modelo 

- `input_shape`: quantas variáveis em cada entrada
- `activation`: função de ativação, vamos usar a `soft_max`

In [12]:
#vamos definir o modelo
model = keras.Sequential()
model.add(Dense(12, input_shape=(12,), activation="softmax"))
model.add(Dense(3, activation="softmax"))

#### Compilando o modelo

Otimizador:
- https://analyticsindiamag.com/guide-to-tensorflow-keras-optimizers/


In [13]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])


#### Treinando o modelo

Usamos 10 epochs

In [14]:
model.fit(X_train, y_train, epochs=8) 

Epoch 1/8
235/235 [==============================] - 1s 1ms/step - loss: 1.0649 - categorical_accuracy: 0.4766
Epoch 2/8
235/235 [==============================] - 0s 1ms/step - loss: 1.0400 - categorical_accuracy: 0.4776
Epoch 3/8
235/235 [==============================] - 0s 1ms/step - loss: 1.0316 - categorical_accuracy: 0.4828
Epoch 4/8
235/235 [==============================] - 0s 1ms/step - loss: 1.0271 - categorical_accuracy: 0.5056
Epoch 5/8
235/235 [==============================] - 0s 2ms/step - loss: 1.0248 - categorical_accuracy: 0.5056
Epoch 6/8
235/235 [==============================] - 0s 1ms/step - loss: 1.0234 - categorical_accuracy: 0.5056
Epoch 7/8
235/235 [==============================] - 0s 1ms/step - loss: 1.0228 - categorical_accuracy: 0.5056
Epoch 8/8
235/235 [==============================] - 0s 1ms/step - loss: 1.0224 - categorical_accuracy: 0.5056


In [15]:
# Avaliacao
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

59/59 - 0s - loss: 1.0350 - categorical_accuracy: 0.4904 - 231ms/epoch - 4ms/step


In [16]:
# previsao
predictions = model.predict(X_test)

59/59 [==============================] - 0s 973us/step


In [17]:
predictions[0]

array([0.30242392, 0.50323963, 0.19433644], dtype=float32)

In [18]:
# transforma dado de [0.85, 0.10,0.05]
# em [-1]
def processa_tendencia (dado):

  lis = []

  for data in dado:

    res = data.index(max(data))
    # Queda
    if res == 0:
      lis += [-1]
    # Estabilidade
    elif res == 1:
      lis += [0]
    # Aumento
    else:
      lis += [1]

  return lis

In [19]:
pred = [list(i) for i in predictions]

pp = processa_tendencia(pred)
pr = processa_tendencia(y_test)

In [21]:
acertos = 0
erros = 0

for i in range(len(pp)):
  if pp[i] == pr[i]:
    acertos += 1
  else:
    erros += 1

print("Em relação ao total de dados:")
print(f"Acertos: {acertos} -> {100*acertos/len(pp):.2f}%")
print(f"Erros: {erros} -> {100*erros/len(pp):.2f}%")

Em relação ao total de dados:
Acertos: 922 -> 49.04%
Erros: 958 -> 50.96%


In [22]:
acertos = 0
erros = 0
queda = 0

for i in range(len(pp)):
  if pr[i] == -1:
    queda +=1
  if pp[i] == pr[i] and pr[i] == -1:
    acertos += 1
  else:
    erros += 1

print("Analisando as quedas:")
print(f"Numero de quedas: {queda} -> {100*queda/len(pp):.2f}%")
print(f"Acertos: {acertos} -> {100*acertos/queda:.2f}%")


Analisando as quedas:
Numero de quedas: 576 -> 30.64%
Acertos: 0 -> 0.00%


In [24]:
acertos = 0
erros = 0
estab = 0

for i in range(len(pp)):
  if pr[i] == 0:
    estab +=1
  if pp[i] == pr[i] and pr[i] == 0:
    acertos += 1
  else:
    erros += 1

print("Analisando a estabilidade:")
print(f"Numero de estabilidades: {estab} -> {100*estab/len(pp):.2f}%")
print(f"Acertos: {acertos} -> {100*acertos/estab:.2f}%")

Analisando a estabilidade:
Numero de estabilidades: 922 -> 49.04%
Acertos: 922 -> 100.00%


In [26]:
acertos = 0
erros = 0
aumento = 0

for i in range(len(pp)):
  if pr[i] == 1:
    aumento +=1
  if pp[i] == pr[i] and pr[i] == 1:
    acertos += 1
  else:
    erros += 1

print("Analisando aumento:")
print(f"Numero de aumento: {aumento} -> {100*aumento/len(pp):.2f}%")
print(f"Acertos: {acertos} -> {100*acertos/aumento:.2f}%")


Analisando aumento:
Numero de aumento: 382 -> 20.32%
Acertos: 0 -> 0.00%
